In [ ]:
%run DemoEspresso_CreateFunction

In [ ]:
from synapse.ml.core.platform import find_secret

# Fill in the following lines with your service information
service_name = "<MyOpenAIService>" # Name of your OpenAI service
deployment_name = "<MyModelDepl>" # Name of your deployment in OpenAI
key = find_secret("<MySecret>", "<MyKeyVault>")  # replace this with your secret and keyvault

In [ ]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

df = spark.read.option('header', 'true') \
                .option('delimiter', '|') \
                .csv('abfss://demodata@adlsespressoopenai.dfs.core.windows.net/insuranceclaims.csv') #Replace with your ADLS account

display(df)

In [ ]:
dfprompt = df.select(col("insuranceaddress"), \
    address_prompt_udf(col("insuranceaddress")).alias("prompt_address"), \
    col("accidentcountry"), \
    country_prompt_udf(col("accidentcountry")).alias("prompt_country"), \
    col("accidentdate"), \
    date_prompt_udf(col("accidentdate")).alias("prompt_date"), \
    col("accidenttime"), \
    time_prompt_udf(col("accidenttime")).alias("prompt_time"))          

display(dfprompt)

In [ ]:
from synapse.ml.cognitive import OpenAICompletion

completion_address = (
    OpenAICompletion()
    .setSubscriptionKey(key)
    .setDeploymentName(deployment_name)
    .setUrl("https://{}.openai.azure.com/".format(service_name))
    .setMaxTokens(2048)
    .setPromptCol("prompt_address")
    .setErrorCol("error_address")
    .setOutputCol("response_address")
)

completion_country = (
    OpenAICompletion()
    .setSubscriptionKey(key)
    .setDeploymentName(deployment_name)
    .setUrl("https://{}.openai.azure.com/".format(service_name))
    .setMaxTokens(2048)
    .setPromptCol("prompt_country")
    .setErrorCol("error_country")
    .setOutputCol("response_country")
)

completion_date = (
    OpenAICompletion()
    .setSubscriptionKey(key)
    .setDeploymentName(deployment_name)
    .setUrl("https://{}.openai.azure.com/".format(service_name))
    .setMaxTokens(2048)
    .setPromptCol("prompt_date")
    .setErrorCol("error_date")
    .setOutputCol("response_date")
)

completion_time = (
    OpenAICompletion()
    .setSubscriptionKey(key)
    .setDeploymentName(deployment_name)
    .setUrl("https://{}.openai.azure.com/".format(service_name))
    .setMaxTokens(2048)
    .setPromptCol("prompt_time")
    .setErrorCol("error_time")
    .setOutputCol("response_time")
)



In [ ]:
from pyspark.sql.functions import col

address_df = completion_address.transform(dfprompt)
country_df = completion_country.transform(address_df)
date_df = completion_date.transform(country_df)
completed_df = completion_time.transform(date_df).cache()

display(completed_df)


In [ ]:
from pyspark.sql.functions import *

display(
    completed_df.select( 
        col('insuranceaddress'), 
        col('response_address.choices.text').getItem(0).alias('address'),   
        col('accidentcountry'), 
        col('response_country.choices.text').getItem(0).alias('country'),  
        col('accidentdate'), 
        col('response_date.choices.text').getItem(0).alias('date'), 
        col('accidenttime'), 
        col('response_time.choices.text').getItem(0).alias('time'),     
    )
    
)